# RSS: Cinco Días

## 1. Import Libraries

In [1]:
import feedparser
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import googletrans
import datetime
from termcolor import colored
from IPython.display import display, Markdown, Latex, HTML
import re
from IPython.display import Image
import urllib
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import numpy as np
from PIL import Image

In [2]:
def get_only_text1(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    # Eliminar Última Hora
    rest = soup.find_all('p', attrs={'class' : "line-clamp_x2 ue-c-widget__article-headline"})[0].text
    index = text.find(rest)
    final_text = text[:index]
    return [final_text,soup.title.text]

def get_only_text(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    return [text,soup.title.text]

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def string_found(string1, string2):
    
    if re.search(r"\b" + re.escape(string1) + r"\b", string2.lower()):
        return True
    return False     
    
def bank_tags(text):
    tags = []
    dict_banks = {'Santander':['santander'],'BBVA':['bbva'],'Bankinter':['bankinter'],'Bankia':['bankia'],
                 'Sabadell':['sabadell'],'ING':['ing'],'Abanca':['abanca'],'Deutsche Bank':['deutsche'],
                  'CaixaBank':['caixabank'],'Openbank':['openbank']}
    for key,values in dict_banks.items():
        for i in values:
            if string_found(i,text.lower()):
                tags.append(key)
                break
    return tags

In [3]:
def cincod_news(cincod_entries,word_count=100):
    cincod = {}
    for i in range(len(cincod_entries)):
        noticia = {'newspaper':None,'title':None,'headline':None,'summarise':None,'date':None,'link':None,'text':None,'current_date':None,'img':None,
                   'premium':None,'tag':None}
        # Periodico
        noticia['newspaper'] = find_between(s=cincod_entries[i]['id'], first='//', last='.' )
        # Title
        noticia['title'] = cincod_entries[i]['title']
        # Headline
        headline =  re.sub(pattern='&quot;',repl='"',string=cincod_entries[i]['summary_detail']['value'])
        noticia['headline'] = headline
        # Date
        noticia['date'] = {'date1':cincod_entries[i]['published'],'date2':cincod_entries[i]['published_parsed']}
        # Link
        noticia['link'] = cincod_entries[i]['id']
        # Text
        text = get_only_text(cincod_entries[i]['id'])[0]
        if text.find('»') != -1:
            index = get_only_text(cincod_entries[i]['id'])[0].index('»') +3
            text = get_only_text(cincod_entries[i]['id'])[0][index:]
        noticia['text'] = text
        # Bank Tags
        noticia['tag'] = bank_tags(text)
        # Summarise
        noticia['summarise'] = summarize(text, word_count=word_count)        
        # Current Date
        noticia['current_date'] = datetime.datetime.now().timetuple()
        # Image
        if len(cincod_entries[i]['links']) > 1:
            noticia['img'] = cincod_entries[i]['links'][1]['href']
        else:
             noticia['img'] = None
        cincod.update({'Noticia'+'_'+str("%02d")%(i+1):noticia})
    return cincod

## 2. RSS Cinco Días

In [4]:
url_cinco_dias = 'https://cincodias.elpais.com/seccion/rss/companias/'
cincod = feedparser.parse(url_cinco_dias)
cincod_entries = cincod['entries']

print('Total Entradas Cinco Días:',len(cincod_entries))

Total Entradas Cinco Días: 27


In [5]:
cincod = cincod_news(cincod_entries,word_count=100)

In [6]:
yesterday = datetime.datetime.today() - datetime.timedelta(days=1)
before_yes = datetime.datetime.today() - datetime.timedelta(days=2)

for i in list(cincod.keys()):
    if cincod[i]['date']['date2'][2] in [datetime.datetime.today().day,yesterday.day,before_yes.day]:
        print(colored((cincod[i]['newspaper']).upper(),'green',attrs=['bold']))
        print(colored(cincod[i]['date']['date1'],'grey',attrs=['bold']))
        print(colored(cincod[i]['title'],'magenta',attrs=['bold']))
        print(colored('Headline','blue'))
        print(cincod[i]['headline'])
        print(colored('Summarise','blue'))
        
        if cincod[i]['summarise'] != 'Premium Content':
            print(cincod[i]['summarise'])
        else:
            print(colored(cincod[i]['summarise'],'red'))
        print(colored('Text','blue'))
        print(cincod[i]['text'])
        url = cincod[i]['img']
        display(Image(url=url,width=500))
        display(Markdown('<a href="'+cincod[i]['link']+'">Link</a>'))
        print('\n')

CINCODIAS
Mon, 20 Jul 2020 21:57:58 +0200
Iberdrola, primera empresa del Ibex por capitalización, tras superar a Inditex
Headline
Cerró la sesión con 71.727,6 millones, frente a los 71.651 millones del grupo textil
Summarise
Iberdrola se convirtió este lunes en la primera empresa del Ibex por capitalización bursátil, tras subir un 1,4% y superar ligeramente a Inditex, que ha sido líder del selectivo en los últimos años.
La compañía energética cerró la sesión con 71.727,6 millones de euros de capitalización, a 11,24 euros por acción, mientras que el grupo textil de Amancio Ortega lo hizo a 71.651 millones, a 22,99 euros por título, incluyendo en ambos casos los derechos de cotización del scrip dividend (títulos para retribuir al accionista).
Text
Iberdrola se convirtió este lunes en la primera empresa del Ibex por capitalización bursátil, tras subir un 1,4% y superar ligeramente a Inditex, que ha sido líder del selectivo en los últimos años. La compañía energética cerró la sesión con 71

TypeError: 'module' object is not callable